In [0]:
!/opt/bin/nvidia-smi

Sat Nov 23 17:35:28 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from keras.layers import Input, Dense, Flatten, Dropout
from keras.models import Model
from keras.datasets import cifar100
from keras.optimizers import Adam, SGD
from keras.utils import np_utils
from keras import backend as K
import numpy as np
import os
from keras.regularizers import l2
import tensorflow as tf
import time
import datetime
import argparse
import datetime
import socket
from sklearn import preprocessing
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import time
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image, ImageOps
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
##### For one-hot label
from keras.utils import np_utils


Using TensorFlow backend.


In [0]:
pip install -U scikit-learn


Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.21.3)


**Load data file. We sperate loading training data and testing. And define variables which can represent file path.**

In [0]:
nb_classes = 397
img_depth = 3
data_dir ='/content/drive/My Drive/Colab Notebooks/SUN_Practice/'+'SUN397'
train_img_file = '/content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/Training_01.txt'   # Training_01.txt
test_img_file = '/content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/Testing_01.txt'     # Testing_01.txt
classes_name_list = '/content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/ClassName.txt'
train_label_file ='/content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/ClassName.txt'
test_label_file = '/content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/ClassName.txt'

train_img_file_path='/content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/Training_01.txt'
test_img_file_path='/content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/Testing_01.txt'


In [4]:
print(train_img_file)

/content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/Training_01.txt


In [5]:
print(classes_name_list)

/content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/ClassName.txt


In [6]:
print('Begin to create a map to tansfer the str label to int label...')

Begin to create a map to tansfer the str label to int label...


In [0]:
class_name_file =  classes_name_list

**Through class_name_file, we can get each class using readlines function.And using split() function to remove '\n'**

In [0]:
class_str = [str(line.strip()) for line in open(class_name_file).readlines()]

In [9]:
print(class_str)

['/a/abbey', '/a/airplane_cabin', '/a/airport_terminal', '/a/alley', '/a/amphitheater', '/a/amusement_arcade', '/a/amusement_park', '/a/anechoic_chamber', '/a/apartment_building/outdoor', '/a/apse/indoor', '/a/aquarium', '/a/aqueduct', '/a/arch', '/a/archive', '/a/arrival_gate/outdoor', '/a/art_gallery', '/a/art_school', '/a/art_studio', '/a/assembly_line', '/a/athletic_field/outdoor', '/a/atrium/public', '/a/attic', '/a/auditorium', '/a/auto_factory', '/b/badlands', '/b/badminton_court/indoor', '/b/baggage_claim', '/b/bakery/shop', '/b/balcony/exterior', '/b/balcony/interior', '/b/ball_pit', '/b/ballroom', '/b/bamboo_forest', '/b/banquet_hall', '/b/bar', '/b/barn', '/b/barndoor', '/b/baseball_field', '/b/basement', '/b/basilica', '/b/basketball_court/outdoor', '/b/bathroom', '/b/batters_box', '/b/bayou', '/b/bazaar/indoor', '/b/bazaar/outdoor', '/b/beach', '/b/beauty_salon', '/b/bedroom', '/b/berth', '/b/biology_laboratory', '/b/bistro/indoor', '/b/boardwalk', '/b/boat_deck', '/b/boat

In [10]:
class_count = len(class_str)
print('%d class names are loaded' % class_count)

397 class names are loaded


begin to create the map

**Using preprocessing.LabelEncoder function to encode labels with value between 0 and n_classes-1.It can be used to transform non-numerical labels (as long as they are hashable and comparable) to numerical labels.**

In [0]:
le = preprocessing.LabelEncoder()

**Fit label encoder**

In [12]:
le.fit(class_str)

LabelEncoder()

In [13]:
print(list(le.classes_))
print('Label map created...')

['/a/abbey', '/a/airplane_cabin', '/a/airport_terminal', '/a/alley', '/a/amphitheater', '/a/amusement_arcade', '/a/amusement_park', '/a/anechoic_chamber', '/a/apartment_building/outdoor', '/a/apse/indoor', '/a/aquarium', '/a/aqueduct', '/a/arch', '/a/archive', '/a/arrival_gate/outdoor', '/a/art_gallery', '/a/art_school', '/a/art_studio', '/a/assembly_line', '/a/athletic_field/outdoor', '/a/atrium/public', '/a/attic', '/a/auditorium', '/a/auto_factory', '/b/badlands', '/b/badminton_court/indoor', '/b/baggage_claim', '/b/bakery/shop', '/b/balcony/exterior', '/b/balcony/interior', '/b/ball_pit', '/b/ballroom', '/b/bamboo_forest', '/b/banquet_hall', '/b/bar', '/b/barn', '/b/barndoor', '/b/baseball_field', '/b/basement', '/b/basilica', '/b/basketball_court/outdoor', '/b/bathroom', '/b/batters_box', '/b/bayou', '/b/bazaar/indoor', '/b/bazaar/outdoor', '/b/beach', '/b/beauty_salon', '/b/bedroom', '/b/berth', '/b/biology_laboratory', '/b/bistro/indoor', '/b/boardwalk', '/b/boat_deck', '/b/boat

# **load training data**

In [14]:
print('\nBegin to load training data...\n')
desired_img_dim=224
print('Loading image file %s' % train_img_file_path)
#train_img_file_path = data_dir + train_img_file
#train_label_file_path = data_dir + train_label_file



Begin to load training data...

Loading image file /content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/Training_01.txt


**read all the image file name**

In [15]:
start_time_ = time.time()
train_img_file_path = [str(line.strip()) for line in open(train_img_file_path).readlines()]  # read all the image file name
nb_sample = len(train_img_file_path)
print('Image count: %d' % nb_sample)

Image count: 19850


**Firstly, we use np.empty to create data_resized_holder which just contain non-meaning value : zero**

In [0]:
data_resized_holder = np.empty([nb_sample, desired_img_dim, desired_img_dim, img_depth], dtype='float32')

In [17]:
data_resized_holder.shape

(19850, 224, 224, 3)

In [20]:
print(data_dir)
print(data_dir+img_file1)

/content/drive/My Drive/Colab Notebooks/SUN_Practice/SUN397
/content/drive/My Drive/Colab Notebooks/SUN_Practice/SUN397/y/youth_hostel/sun_ammfvvfoiqxeovyl.jpg


In [19]:
for idx in range(nb_sample):
    img_file1 = train_img_file_path[idx].replace("\\", "/")     # the image file path
    print(img_file1)

/a/abbey/sun_aqswjsnjlrfzzhiz.jpg
/a/abbey/sun_aaoktempcmudsvna.jpg
/a/abbey/sun_arxtvfkkycnqhfkj.jpg
/a/abbey/sun_axcfsjajikefppey.jpg
/a/abbey/sun_arsnqaftitwcpsog.jpg
/a/abbey/sun_aizsoszlxxvbjhqd.jpg
/a/abbey/sun_absjqmjicjjmurwn.jpg
/a/abbey/sun_acvjxpbfdkwcyyga.jpg
/a/abbey/sun_akighlforrjygtkv.jpg
/a/abbey/sun_aakbdcgfpksytcwj.jpg
/a/abbey/sun_aqyoszxqzenqukbn.jpg
/a/abbey/sun_aesfrdpuprlhzzss.jpg
/a/abbey/sun_anunhwbsxmmdzwmk.jpg
/a/abbey/sun_aamvxnvouicstkjb.jpg
/a/abbey/sun_asvrsxpizfhvuftq.jpg
/a/abbey/sun_ajgccggyendxydwa.jpg
/a/abbey/sun_atghahjqptwxcnbu.jpg
/a/abbey/sun_afuhbsqqwerjshqj.jpg
/a/abbey/sun_acguwqihawgzdnju.jpg
/a/abbey/sun_aggditxsglyqgqsk.jpg
/a/abbey/sun_alqgvwyohunprcsh.jpg
/a/abbey/sun_axcwnadpsaegsupw.jpg
/a/abbey/sun_argijfrtajgytjrs.jpg
/a/abbey/sun_ayahbvkpizprwkbw.jpg
/a/abbey/sun_awbmttvnojoatvva.jpg
/a/abbey/sun_akkyidsxmnaeupuv.jpg
/a/abbey/sun_ayqybhyaetvpnyts.jpg
/a/abbey/sun_atbseznnhjnnmefq.jpg
/a/abbey/sun_auobgxxzizpqpupj.jpg
/a/abbey/sun_a

**We get the img_file1 and then use this file path we can load image,get each image and resize it to our expected dimension ,and then give to the holder**

In [0]:

for idx in range(nb_sample):
    img_file1 = train_img_file_path[idx].replace("\\", "/")     # the image file path
    # print(str(img_file1))
    # 1. read the image
    img1 = image.load_img(data_dir+img_file1)

    # 2. resize
    img1 = img1.resize((desired_img_dim, desired_img_dim), resample=0)


    # 6. give to the holder
    data_resized_holder[idx] = img1
    if(idx % 1000==0):
        print('%d image loaded.' % idx)

print('\nImage file loaded, the shape is ' + str(data_resized_holder.shape))


Image count: 19850
0 image loaded.
1000 image loaded.
2000 image loaded.
3000 image loaded.
4000 image loaded.
5000 image loaded.
6000 image loaded.
7000 image loaded.
8000 image loaded.
9000 image loaded.
10000 image loaded.


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1385478 bytes but only got 23143. Skipping tag 513
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 7276331008 bytes but only got 0. Skipping tag 7
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1879769088 bytes but only got 0. Skipping tag 4
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3014066176 bytes but only got 0. Skipping tag 4
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  

11000 image loaded.
12000 image loaded.
13000 image loaded.
14000 image loaded.
15000 image loaded.
16000 image loaded.


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3145728 bytes but only got 0. Skipping tag 19788
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 80000 bytes but only got 0. Skipping tag 64640
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2684485632 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5469503488 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Exp

17000 image loaded.
18000 image loaded.
19000 image loaded.

Image file loaded, the shape is (19850, 224, 224, 3)


In [0]:
x_train.shape

(0,)

**loading the training labels text**

In [23]:
one_hot = True
print('Loading label file %s' % train_label_file)
label_str = [str(line.strip()) for line in open(train_label_file).readlines()]

Loading label file /content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/ClassName.txt


In [24]:
print(label_str)

['/a/abbey', '/a/airplane_cabin', '/a/airport_terminal', '/a/alley', '/a/amphitheater', '/a/amusement_arcade', '/a/amusement_park', '/a/anechoic_chamber', '/a/apartment_building/outdoor', '/a/apse/indoor', '/a/aquarium', '/a/aqueduct', '/a/arch', '/a/archive', '/a/arrival_gate/outdoor', '/a/art_gallery', '/a/art_school', '/a/art_studio', '/a/assembly_line', '/a/athletic_field/outdoor', '/a/atrium/public', '/a/attic', '/a/auditorium', '/a/auto_factory', '/b/badlands', '/b/badminton_court/indoor', '/b/baggage_claim', '/b/bakery/shop', '/b/balcony/exterior', '/b/balcony/interior', '/b/ball_pit', '/b/ballroom', '/b/bamboo_forest', '/b/banquet_hall', '/b/bar', '/b/barn', '/b/barndoor', '/b/baseball_field', '/b/basement', '/b/basilica', '/b/basketball_court/outdoor', '/b/bathroom', '/b/batters_box', '/b/bayou', '/b/bazaar/indoor', '/b/bazaar/outdoor', '/b/beach', '/b/beauty_salon', '/b/bedroom', '/b/berth', '/b/biology_laboratory', '/b/bistro/indoor', '/b/boardwalk', '/b/boat_deck', '/b/boat

**Transform Categories Into Integers**

In [0]:
nb_unique = len(label_str)
labels_unique = le.transform(label_str)

In [26]:
print(nb_unique)

397


In [27]:
print(labels_unique)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

**Stack arrays in sequence horizontally (column wise).And choose each label 50 times! in each class**

In [28]:
labels_holder = np.hstack((  [ labels_unique[i] ] * 50 for i in range(nb_unique)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


In [29]:
list([ labels_unique[i] ] * 50 for i in range(nb_unique))

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 [2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2],
 [3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  

In [30]:
print(labels_holder)

[  0   0   0 ... 396 396 396]


**get training data labels**

In [100]:
nb_sample = len(labels_holder)
if one_hot == True:
    labels = np.array([[float(i == l) for i in range(nb_classes)] for l in labels_holder])
else:
    labels = labels_holder
print('Labels loaded, shape is:' + str(labels.shape))



Labels loaded, shape is:(19850, 397)


**loading training data**

In [0]:
x_train, nb_train_sample_1 = data_resized_holder, nb_sample

In [0]:
y_train, nb_train_sample_2 = labels, nb_sample

In [0]:
del data_resized_holder
del labels

In [0]:
x_train.shape

(19850, 224, 224, 3)

In [0]:
y_train.shape

(19850, 397)

# **Saving train data and test data**

**Saving x_train y_train as .npy  file**

In [0]:
np.save('/content/drive/My Drive/Colab Notebooks/SUN_Practice/x_tain.npy',x_train)

In [0]:
np.save('/content/drive/My Drive/Colab Notebooks/SUN_Practice/y_train.npy',y_train)

In [0]:
del x_train
#del y_train


# **loading testing data**

**load testing data**

**read all the image file name**

In [31]:
print('Loading image file %s' % test_img_file_path )
start_time_ = time.time()
test_img_file_path = [str(line.strip()) for line in open(test_img_file_path).readlines()]  # read all the image file name
nb_sample = len(test_img_file_path)
print('Image count: %d' % nb_sample)


Loading image file /content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/Testing_01.txt
Image count: 19850


**Firstly, we use np.empty to create data_resized_holder which just contain non-meaning value : zero**

In [0]:
data_resized_holder = np.empty([nb_sample, desired_img_dim, desired_img_dim, img_depth], dtype='float32')

In [33]:
data_resized_holder.shape

(19850, 224, 224, 3)

In [34]:
for idx in range(nb_sample):
    img_file1 = data_dir + test_img_file_path[idx].replace("\\", "/")     # the image file path
    print(str(img_file1))


/content/drive/My Drive/Colab Notebooks/SUN_Practice/SUN397/a/abbey/sun_ajkqrqitspwywirx.jpg
/content/drive/My Drive/Colab Notebooks/SUN_Practice/SUN397/a/abbey/sun_ajhtswxgrqbeiikc.jpg
/content/drive/My Drive/Colab Notebooks/SUN_Practice/SUN397/a/abbey/sun_ahebfjooupcqbjht.jpg
/content/drive/My Drive/Colab Notebooks/SUN_Practice/SUN397/a/abbey/sun_amdwiijwnagslgef.jpg
/content/drive/My Drive/Colab Notebooks/SUN_Practice/SUN397/a/abbey/sun_awzqxzqukmbujwzm.jpg
/content/drive/My Drive/Colab Notebooks/SUN_Practice/SUN397/a/abbey/sun_aupohozmrmtfgcsw.jpg
/content/drive/My Drive/Colab Notebooks/SUN_Practice/SUN397/a/abbey/sun_alwjgbdpxuskytjo.jpg
/content/drive/My Drive/Colab Notebooks/SUN_Practice/SUN397/a/abbey/sun_autcnwvjhdroaklu.jpg
/content/drive/My Drive/Colab Notebooks/SUN_Practice/SUN397/a/abbey/sun_akgjeakhepssnilh.jpg
/content/drive/My Drive/Colab Notebooks/SUN_Practice/SUN397/a/abbey/sun_awmdgbfmljliozsj.jpg
/content/drive/My Drive/Colab Notebooks/SUN_Practice/SUN397/a/abbey/su

**read the image**

In [0]:
for idx in range(nb_sample):
    img_file1 = data_dir + test_img_file_path[idx].replace("\\", "/")     # the image file path
    # print(str(img_file1))
    # 1. read the image
    img1 = image.load_img(img_file1)

    # 2. resize
    img1 = img1.resize((desired_img_dim, desired_img_dim), resample=0)


    # 6. give to the holder
    data_resized_holder[idx] = img1
    if(idx % 1000==0):
        print('%d image loaded.' % idx)

print('\nImage file loaded, the shape is ' + str(data_resized_holder.shape))


Loading image file /content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/Testing_01.txt
Image count: 19850
0 image loaded.


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


1000 image loaded.
2000 image loaded.
3000 image loaded.


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4718592 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))


4000 image loaded.
5000 image loaded.
6000 image loaded.
7000 image loaded.
8000 image loaded.


...

11000 image loaded.
12000 image loaded.
13000 image loaded.
14000 image loaded.


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1441814 bytes but only got 0. Skipping tag 33
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 428 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 196608 bytes but only got 0. Skipping tag 60
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 196608 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5

15000 image loaded.
16000 image loaded.


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2621440 bytes but only got 0. Skipping tag 19788
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 80000 bytes but only got 0. Skipping tag 64640
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2684485632 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5469503488 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Exp

17000 image loaded.
18000 image loaded.
19000 image loaded.

Image file loaded, the shape is (19850, 224, 224, 3)


**Load test labels text**

In [35]:
one_hot = True
# loading the training labels
print('Loading label file %s' % test_label_file)
label_str = [str(line.strip()) for line in open(test_label_file).readlines()]

Loading label file /content/drive/My Drive/Colab Notebooks/SUN_Practice/Partitions/ClassName.txt


In [36]:
print(label_str)

['/a/abbey', '/a/airplane_cabin', '/a/airport_terminal', '/a/alley', '/a/amphitheater', '/a/amusement_arcade', '/a/amusement_park', '/a/anechoic_chamber', '/a/apartment_building/outdoor', '/a/apse/indoor', '/a/aquarium', '/a/aqueduct', '/a/arch', '/a/archive', '/a/arrival_gate/outdoor', '/a/art_gallery', '/a/art_school', '/a/art_studio', '/a/assembly_line', '/a/athletic_field/outdoor', '/a/atrium/public', '/a/attic', '/a/auditorium', '/a/auto_factory', '/b/badlands', '/b/badminton_court/indoor', '/b/baggage_claim', '/b/bakery/shop', '/b/balcony/exterior', '/b/balcony/interior', '/b/ball_pit', '/b/ballroom', '/b/bamboo_forest', '/b/banquet_hall', '/b/bar', '/b/barn', '/b/barndoor', '/b/baseball_field', '/b/basement', '/b/basilica', '/b/basketball_court/outdoor', '/b/bathroom', '/b/batters_box', '/b/bayou', '/b/bazaar/indoor', '/b/bazaar/outdoor', '/b/beach', '/b/beauty_salon', '/b/bedroom', '/b/berth', '/b/biology_laboratory', '/b/bistro/indoor', '/b/boardwalk', '/b/boat_deck', '/b/boat

**Transform Categories Into Integers**

In [0]:
nb_unique = len(label_str)
labels_unique = le.transform(label_str)

In [38]:
print(labels_unique)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

**Stack arrays in sequence horizontally (column wise).And choose each label 50 times! in each class**

In [40]:
labels_holder = np.hstack((  [ labels_unique[i] ] * 50 for i in range(nb_unique)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


In [41]:
print(labels_holder)

[  0   0   0 ... 396 396 396]


**get training data labels**

In [42]:

nb_sample = len(labels_holder)
if one_hot == True:
    labels = np.array([[float(i == l) for i in range(nb_classes)] for l in labels_holder])
else:
    labels = labels_holder
print('Labels loaded, shape is:' + str(labels.shape))



Labels loaded, shape is:(19850, 397)


**Loading testing data**

In [0]:
x_test, nb_test_sample_1 = data_resized_holder, nb_sample

In [0]:
y_test, nb_test_sample_2 = labels, nb_sample

In [0]:
del data_resized_holder
del labels

**Saving x_test y_test .npy file**

In [0]:
np.save('/content/drive/My Drive/Colab Notebooks/SUN_Practice/x_test.npy',x_test)

In [0]:
np.save('/content/drive/My Drive/Colab Notebooks/SUN_Practice/y_test.npy',y_test)

In [0]:
del x_test
del y_test

In [0]:
y_test.shape

(19850, 397)

# **Load train data and test data**

**Load train data**

In [0]:
x_train = np.load('/content/drive/My Drive/Colab Notebooks/SUN_Practice/x_tain.npy')

In [0]:
y_train = np.load('/content/drive/My Drive/Colab Notebooks/SUN_Practice/y_test.npy')

In [0]:
x_train.shape

(19850, 224, 224, 3)

In [0]:
print(y_train[0])

0


**Load test data**

In [0]:
x_test = np.load('/content/drive/My Drive/Colab Notebooks/SUN_Practice/x_test.npy')

In [0]:
y_test = np.load('/content/drive/My Drive/Colab Notebooks/SUN_Practice/y_test.npy')

In [0]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

# **Data Auguation**

In [0]:
generator = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


# **Using VGG16 train this dataset**

In [0]:
import os
import keras
from keras.models import Model
from keras.layers import Dense,Flatten,Dropout
from keras import datasets
from keras.applications.vgg16 import VGG16
#from vgg16_places_365 import VGG16_Places365 
from keras.optimizers import SGD, Adam
from keras import optimizers
from keras.callbacks import EarlyStopping

In [0]:
pretrain_model_vgg = VGG16(include_top=False, weights='imagenet', input_shape=(dim, dim, channel))

for layer in pretrain_model_vgg.layers:
    layer.trainable = False
    
model = Flatten()(pretrain_model_vgg.output)
model = Dense(4096, activation="relu")(model)
model = Dense(397, activation="relu")(model)
model = Dropout(0.3)(model)
model = Dense(num_classes, activation="softmax")(model)
pretrain_model = Model(pretrain_model_vgg.input, model, name='pretrain_vgg_16')
pretrain_model.summary()

In [0]:
learning_rate = 0.05
batch_size=64
Epoch = 10
decay_rate = learning_rate / Epoch
momentum = 0.9
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
# sgd=SGD(lr=0.05,decay=1e-5)
#opt= Adam(lr=0.01)


In [0]:
pretrain_model.save_weights('model_sun_initial.h5')

In [0]:
training_runs = []
for i in range(3):
    #model_updated.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
  

    pretrain_model.compile(loss='mean_squared_error', optimizer=optimizers.adam(lr=0.0001), metrics=['accuracy'])
    keras.callbacks.LearningRateScheduler(learning_rate_schedule)

    history = pretrain_model.fit_generator(generator.flow(x_train, y_train.values, batch_size=batch_size),
                    steps_per_epoch=len(x_train) / batch_size, 
                    epochs=10,
                    verbose=1,
                    shuffle=True ,
                    validation_data=(x_test, y_test.values))
    
   
    training_runs.append(history)
    model_updated.get_weights()
    if i == 2:
        pretrain_model.save_weights('model1.h5')
    else:
        pretrain_model.load_weights('model_sun_initial.h5')
    print()

In [0]:
#hist = pretrain_model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = Epoch, batch_size = 10)

In [0]:
import matplotlib.pyplot as plt
import numpy as np

plt.subplot(2, 3, 1)
plt.plot(training_runs[0].history['acc'])
plt.legend(['train'], loc='lower right')

plt.subplot(2, 3, 2)
plt.plot(training_runs[1].history['acc'])
plt.legend(['train'], loc='lower right')

plt.subplot(2, 3, 3)
plt.plot(training_runs[2].history['acc'])
plt.legend(['train'], loc='lower right')

plt.subplot(2, 3, 4)
plt.plot(training_runs[0].history['val_acc'])
plt.legend(['test'], loc='lower right')

plt.subplot(2, 3, 5)
plt.plot(training_runs[1].history['val_acc'])
plt.legend(['test'], loc='lower right')

plt.subplot(2, 3, 6)
plt.plot(training_runs[2].history['val_acc'])
plt.legend(['test'], loc='lower right')

plt.tight_layout() 

plt.show()

print("Average training accuracy: {}".format(np.mean([training_runs[0].history['acc'][-1], 
                                                     training_runs[1].history['acc'][-1], training_runs[2].history['acc'][-1]])))
print("Average testing accuracy: {}".format(np.mean([training_runs[0].history['val_acc'][-1], 
                                                     training_runs[1].history['val_acc'][-1], training_runs[2].history['val_acc'][-1]])))

# **New Section**

In [0]:
def load_img_file(file, desired_img_dim):
    print('Loading image file %s' % file)
    start_time_ = time.time()
    img_file_path = [str(line.strip()) for line in open(file).readlines()]  # read all the image file name
    nb_sample = len(img_file_path)
    print('Image count: %d' % nb_sample)

    data_resized_holder = np.empty([nb_sample, desired_img_dim, desired_img_dim,img_depth], dtype='float32')

    for idx in range(nb_sample):
        img_file1 = self.data_dir + img_file_path[idx].replace("\\", "/")     # the image file path
        # print(str(img_file1))
        # 1. read the image
        img1 = image.load_img(img_file1)

        # 2. resize
        img1 = img1.resize((desired_img_dim, desired_img_dim), resample=0)


        # 6. give to the holder
        data_resized_holder[idx] = img1
        if(idx % 1000==0):
            print('%d image loaded.' % idx)

    print('\nImage file loaded, the shape is ' + str(data_resized_holder.shape))

    return data_resized_holder, nb_sample



In [0]:
def load_label_file(file, one_hot=True):
    # loading the training labels
    print('Loading label file %s' % file)
    label_str = [str(line.strip()) for line in open(file).readlines()]
    nb_unique = len(label_str)
    labels_unique = self.le.transform(label_str)
    # print(labels_unique)
    labels_holder = np.hstack((  [ labels_unique[i] ] * 50 for i in range(nb_unique)))
    # print(labels_holder)
    nb_sample = len(labels_holder)
    if one_hot == True:
        labels = np.array([[float(i == l) for i in range(nb_classes)] for l in labels_holder])
    else:
        labels = labels_holder
    print('Labels loaded, shape is:' + str(labels.shape))


    return labels, nb_sample



In [0]:
def load(desired_image_dim, one_hot= True):

    ############################################ loading training data##################################################
    print('\nBegin to load training data...\n')
    train_img_file_path = data_dir + train_img_file
    train_label_file_path = data_dir + train_label_file

    x_train, nb_train_sample_1 = load_img_file(train_img_file_path, desired_image_dim)
    y_train, nb_train_sample_2 = load_label_file(train_label_file_path, one_hot=one_hot)
    assert nb_train_sample_1 == nb_train_sample_2, "The train img and label samples are not matched..."
    nb_train_sample = nb_train_sample_1

    print('\nTraining data loaded.\n')

    ############################################ loading testing data##################################################
    print('\nBegin to load testing data...\n')
    test_img_file_path = data_dir + test_img_file
    test_label_file_path = data_dir + test_label_file

    x_test, nb_test_sample_1 = load_img_file(test_img_file_path, desired_image_dim)
    y_test, nb_test_sample_2 = load_label_file(test_label_file_path, one_hot=one_hot)
    assert nb_test_sample_1 == nb_test_sample_2, "The test img and label samples are not matched..."
    self.nb_test_sample = nb_test_sample_1

    print('\nTesting data loaded.\n')

    return x_train, y_train, nb_train_sample, x_test, y_test, nb_test_sample



In [0]:
def load_set(desired_image_dim, nb_tr_per_class=50, one_hot=True):

    x_train, y_train, nb_tr_sample, x_test, y_test, nb_te_sample = load(desired_image_dim, one_hot=one_hot)

    return x_train, y_train, nb_train_sample, x_test, y_test, nb_test_sample



In [0]:
sun397 = SUN397(data_dir)
sun397.load_set(desired_image_dim=224, one_hot=False)


NameError: ignored

In [0]:
print('Loading image file %s' % file)
        start_time_ = time.time()
        img_file_path = [str(line.strip()) for line in open(file).readlines()]  # read all the image file name
        nb_sample = len(img_file_path)
        print('Image count: %d' % nb_sample)

        data_resized_holder = np.empty([nb_sample, desired_img_dim, desired_img_dim, self.img_depth], dtype='float32')

        for idx in range(nb_sample):
            img_file1 = self.data_dir + img_file_path[idx].replace("\\", "/")     # the image file path
            # print(str(img_file1))
            # 1. read the image
            img1 = image.load_img(img_file1)

            # 2. resize
            img1 = img1.resize((desired_img_dim, desired_img_dim), resample=0)


            # 6. give to the holder
            data_resized_holder[idx] = img1
            if(idx % 1000==0):
                print('%d image loaded.' % idx)

        print('\nImage file loaded, the shape is ' + str(data_resized_holder.shape))


In [0]:
data_dir

In [0]:
import numpy as np
import os
import cv2
import keras
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
print(DATASET_PATH+str('/') +str(folders[folder])+str('/'))

/content/drive/My Drive/Colab Notebooks/SUN_Practice/SUN397/a/


In [0]:

dim = 224
num_classes = 397
channel = 3 

DATASET_PATH = '/content/drive/My Drive/Colab Notebooks/SUN_Practice/SUN397'
one_hot_lookup = np.eye(num_classes)

temp_x = []
temp_y = []
test_x = []
test_label = []
train_x = []
train_label = []

os.chdir(DATASET_PATH)
folders = os.listdir()  #List different 15 classes
for folder in range(len(folders)):
    folder_paths = DATASET_PATH+str('/') +str(folders[folder])+str('/')
    os.chdir(folder_paths)
    image_in_folder = os.listdir()
    print(str(folder)+" : " + str(folders[folder]))
    for image in range(len(image_in_folder)):
        img = cv2.imread(image_in_folder[image],cv2.IMREAD_COLOR)
        img = cv2.resize(img, (dim,dim))
        temp_x.append(np.reshape(img, (dim,dim, channel)))
        temp_y.append(np.reshape(one_hot_lookup[int(folder)], [num_classes]))   
    temp_x = np.array(temp_x)
    temp_y = np.array(temp_y)
    p = np.random.permutation(len(temp_x))
    temp_x = temp_x[p]
    temp_y = temp_y[p]

    train_x.extend(temp_x[:50])
    train_label.extend(temp_y[:100])
    test_x.extend(temp_x[50:])
    test_label.extend(temp_y[100:])    
    
    temp_x = []
    temp_y = []


# dataset_x = []
# dataset_y = []

# counter = 0
# for category in sorted(os.listdir(DATASET_PATH)):

#     for fname in os.listdir(DATASET_PATH+"/"+category):
#         counter = counter+1
#         img = cv2.imread(DATASET_PATH+"/"+category+'/'+fname, 2)
#         img = cv2.cvtColor(cv2.resize(img, (dim,dim)), cv2.COLOR_GRAY2BGR)
        
#         dataset_x.append(np.reshape(img, [dim,dim,channel]))
#         dataset_y.append(np.reshape(one_hot_lookup[int(category)], [num_classes]))
#     print("this is counter: " + str(counter))


# dataset_x = np.array(dataset_x)
# dataset_y = np.array(dataset_y)

# # """shuffle dataset"""
# p = np.random.permutation(len(dataset_x))
# dataset_x = dataset_x[p]
# dataset_y = dataset_y[p]
        
# test_x = dataset_x[:int(len(dataset_x)/10)]
# test_y = dataset_y[:int(len(dataset_x)/10)]
# train_x = dataset_x[int(len(dataset_x)/10):]
# train_y = dataset_y[int(len(dataset_x)/10):]

# train_y2 = keras.utils.to_categorical(train_y).astype('float32')
# test_y2 = keras.utils.to_categorical(test_y).astype('float32')
# /content/drive/My Drive/Colab Notebooks/SUN_Practice/SUN397/a

0 : a


error: ignored

In [0]:
train_x = np.array(train_x)
train_label = np.array(train_label)
test_x = np.array(test_x)
test_label = np.array(test_label)    

# Shuffle the training data with the sequence train_x
p_train = np.random.permutation(len(train_x))
train_x = train_x[p_train]
train_label = train_label[p_train]

# Shuffle the testing data with the sequence test_x
p_test = np.random.permutation(len(test_x))
test_x = test_x[p_test]
test_label = test_label[p_test]
